<a href="https://www.kaggle.com/code/nur988/nlp-basic-approach?scriptVersionId=93116331" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB


corpus = [
 "hello, how are you?",
 "im getting bored at home. And you? What do you think?",
 "did you know about counts",
 "let's see if this works!",
 "YES!!!!"
]

ctv=CountVectorizer(tokenizer=word_tokenize,token_pattern=None)
ctv.fit(corpus)



CountVectorizer(token_pattern=None,
                tokenizer=<function word_tokenize at 0x7f9771ff6cb0>)

In [2]:
corpus_transformed=ctv.transform(corpus)
print(corpus_transformed)

  (0, 2)	1
  (0, 4)	1
  (0, 7)	1
  (0, 14)	1
  (0, 16)	1
  (0, 27)	1
  (1, 3)	1
  (1, 4)	2
  (1, 6)	1
  (1, 8)	1
  (1, 9)	1
  (1, 12)	1
  (1, 13)	1
  (1, 15)	1
  (1, 18)	1
  (1, 22)	1
  (1, 24)	1
  (1, 27)	2
  (2, 5)	1
  (2, 10)	1
  (2, 11)	1
  (2, 19)	1
  (2, 27)	1
  (3, 0)	1
  (3, 1)	1
  (3, 17)	1
  (3, 20)	1
  (3, 21)	1
  (3, 23)	1
  (3, 25)	1
  (4, 0)	4
  (4, 26)	1


In [3]:
print(ctv.vocabulary_)

{'hello': 14, ',': 2, 'how': 16, 'are': 7, 'you': 27, '?': 4, 'im': 18, 'getting': 13, 'bored': 9, 'at': 8, 'home': 15, '.': 3, 'and': 6, 'what': 24, 'do': 12, 'think': 22, 'did': 11, 'know': 19, 'about': 5, 'counts': 10, 'let': 20, "'s": 1, 'see': 21, 'if': 17, 'this': 23, 'works': 25, '!': 0, 'yes': 26}


In [4]:
df=pd.read_csv("../input/nlp-getting-started/train.csv")
df_test=pd.read_csv("../input/nlp-getting-started/test.csv")


In [5]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
y=df.text.values

In [7]:
df['fold']=-1
kf=model_selection.StratifiedKFold(n_splits=5)


In [8]:
for fold,(trn,val) in enumerate(kf.split(df,y)):
    df.loc[val,'fold']=fold
    

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [9]:
for fold in range(5):
    train_df=df[df.fold!=fold].reset_index(drop=True)
    test_df=df[df.fold==fold].reset_index(drop=True)
    vect=TfidfVectorizer(tokenizer=word_tokenize,token_pattern=None)
    x_train=vect.fit_transform(train_df.text)
    x_test=vect.transform(test_df.text)
    model=linear_model.LogisticRegression()
    model.fit(x_train,train_df.target)
    preds=model.predict(x_test)
    acc=metrics.accuracy_score(test_df.target,preds)
    print(f"Accuracy Score--{acc}")
    

Accuracy Score--0.81483913328956
Accuracy Score--0.7872619829284307
Accuracy Score--0.7734734077478661
Accuracy Score--0.8035479632063075
Accuracy Score--0.778580814717477


# Inference

In [10]:
vect=TfidfVectorizer(tokenizer=word_tokenize,token_pattern=None)
x_train=vect.fit_transform(df.text)
model=linear_model.LogisticRegression()
model.fit(x_train,df.target)
test=vect.transform(df_test.text)
preds=model.predict(test)



In [11]:
submission=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
submission['target']=preds
submission.to_csv('submission.csv',index=False)